In [9]:
import nltk
nltk.download('gutenberg')

from nltk.corpus import gutenberg
import pandas as pd
import tensorflow as tf


data = gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt', 'w') as f:
    f.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\adhri\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


## Data Preprocessing

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences  
from sklearn.model_selection import train_test_split

with open('hamlet.txt', 'r') as f:
    text = f.read().lower()

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

In [2]:
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [3]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [4]:
len(input_sequences)

25732

In [5]:
max_sequence_length = max([len(x) for x in input_sequences])

In [6]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

In [7]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [10]:
X,y = input_sequences[:,:-1],input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Training the model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_2 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_1 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               100400    
                                                                 
 dense_1 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(GRU(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [17]:
history = model.fit(X_train, y_train, epochs=50, verbose=1,callbacks=[early_stopping])


Epoch 1/50
644/644 [==============================] - 23s 35ms/step - loss: 6.4028 - accuracy: 0.0441
Epoch 2/50
644/644 [==============================] - 21s 32ms/step - loss: 6.2440 - accuracy: 0.0499
Epoch 3/50
644/644 [==============================] - 19s 29ms/step - loss: 6.0953 - accuracy: 0.0528
Epoch 4/50
644/644 [==============================] - 19s 30ms/step - loss: 5.9476 - accuracy: 0.0591
Epoch 5/50
644/644 [==============================] - 19s 29ms/step - loss: 5.8051 - accuracy: 0.0693
Epoch 6/50
644/644 [==============================] - 19s 30ms/step - loss: 5.6655 - accuracy: 0.0768
Epoch 7/50
644/644 [==============================] - 16s 25ms/step - loss: 5.5312 - accuracy: 0.0846
Epoch 8/50
644/644 [==============================] - 22s 35ms/step - loss: 5.4110 - accuracy: 0.0913
Epoch 9/50
644/644 [==============================] - 22s 35ms/step - loss: 5.2904 - accuracy: 0.0988
Epoch 10/50
644/644 [==============================] - 21s 33ms/step - loss: 5.176

In [18]:
def predict_next_word(model,tokenizer,text,max_sequence_length):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_length:
        token_list = token_list[-(max_sequence_length-1):]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    for word,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return ''

predict_next_word(model,tokenizer,'to be or not to be',max_sequence_length)

'buried'

In [19]:
model.save("next_word_lstm.h5")

c:\Users\adhri\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
import pickle
with open('tokenizer.pickle','wb') as file:
  pickle.dump(tokenizer,file,protocol=pickle.HIGHEST_PROTOCOL)